Disclaimer

> Το παρόν notebook αποτελεί τεχνική τεκμηρίωση της μεθοδολογίας δυναμικής τιμολόγησης bundles με χρήση AI μοντέλων και εκτίμησης ελαστικότητας ανά κατηγορία.
> Δεν είναι σχεδιασμένο για άμεση εκτέλεση καθώς:
 - Δεν περιλαμβάνονται τα αρχικά datasets (`orders_df_copy`, `inventory_df`, bundles κ.λπ.)
 - Ορισμένες μεταβλητές και βοηθητικές συναρτήσεις θεωρούνται ήδη ορισμένες
 - Απαιτείται προσαρμογή στον εκάστοτε χώρο εργασίας (paths, στήλες, μοντέλα)

> Στόχος είναι να παρουσιαστεί η λογική, η μεθοδολογία και η δομή της λύσης.

Απαραίτητα Δεδομένα (Είσοδος)
1.  Ιστορικά Δεδομένα Παραγγελιών – orders_df_copy
    Στήλες (SKU, Item Tittle, Category, Brand, FinalUnitPrice, Date)
2. Απόθεμα Προϊόντων – inventory_df
    Στήλες (SKU, Quantity)
3. Bundles – (volume, rule-based, thematic, clearance)


Εκτίμηση ελαστικότητας ανά κατηγορία

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd

category_elasticities = []

for category in orders_df_copy['Category'].dropna().unique():
    df_cat = orders_df_copy[orders_df_copy['Category'] == category]
    
    if df_cat['FinalUnitPrice'].nunique() < 2 or len(df_cat) < 20:
        continue

    X = df_cat[['FinalUnitPrice']]
    y = df_cat['Quantity']
    
    model = LinearRegression()
    model.fit(X, y)
    
    elasticity = model.coef_[0]
    category_elasticities.append({
        'Category': category,
        'EstimatedElasticity': round(elasticity, 3),
        'Observations': len(df_cat)
    })

elasticity_by_category_df = pd.DataFrame(category_elasticities)


Συγκέντρωση πληροφοριών ανά SKU (μέση τιμή, συνολική ζήτηση και αντιστοίχιση κατηγορίας.)

In [ ]:
sku_summary = orders_df_copy.groupby('SKU').agg({
    'FinalUnitPrice': 'mean',
    'Quantity': 'sum'
}).reset_index()

sku_summary.columns = ['SKU', 'AvgPrice', 'TotalUnitsSold']

# Προσθήκη κατηγορίας ανά SKU
sku_category = orders_df_copy[['SKU', 'Category']].drop_duplicates()
sku_summary = pd.merge(sku_summary, sku_category, on='SKU', how='left')

# Προσθήκη ελαστικότητας ανά κατηγορία
sku_summary = pd.merge(sku_summary, elasticity_by_category_df, on='Category', how='left')
sku_summary['EstimatedElasticity'] = sku_summary['EstimatedElasticity'].fillna(-1.5)  # fallback


Υπολογισμός βέλτιστης τιμής ανά SKU --> μεγιστοποίηση εισόδου

In [ ]:
from scipy.optimize import minimize_scalar

def demand_model(price, base_price, base_demand, elasticity):
    pct_change = (price - base_price) / base_price
    return base_demand * (1 + elasticity * -pct_change)

def revenue_function(price, base_price, base_demand, elasticity):
    return -price * demand_model(price, base_price, base_demand, elasticity)

optimized_prices = []

for _, row in sku_summary.iterrows():
    sku = row['SKU']
    base_price = row['AvgPrice']
    base_demand = row['TotalUnitsSold']
    elasticity = row['EstimatedElasticity']

    result = minimize_scalar(
        revenue_function,
        bounds=(base_price * 0.5, base_price * 1.5),
        args=(base_price, base_demand, elasticity),
        method='bounded'
    )

    optimal_price = round(result.x, 2)
    optimized_prices.append({
        'SKU': sku,
        'BasePrice': round(base_price, 2),
        'BaseDemand': int(base_demand),
        'Category': row['Category'],
        'Elasticity': round(elasticity, 3),
        'OptimizedPrice': optimal_price
    })

optimized_df = pd.DataFrame(optimized_prices)


Εφαρμογή στις Bundles με mapping

In [ ]:
sku_price_map = dict(zip(optimized_df['SKU'], optimized_df['OptimizedPrice']))

rule_bundles_df['Price_A'] = rule_bundles_df['SKU_A'].map(sku_price_map)
rule_bundles_df['Price_B'] = rule_bundles_df['SKU_B'].map(sku_price_map)
rule_bundles_df['OptimizedBundlePrice'] = rule_bundles_df['Price_A'] + rule_bundles_df['Price_B']

volume_bundles['OptimizedUnitPrice'] = volume_bundles['SKU'].map(sku_price_map)
volume_bundles['OptimizedBundlePrice'] = volume_bundles['OptimizedUnitPrice'] * volume_bundles['Qty']

clearance_bundles_df['Price_A'] = clearance_bundles_df['SKU_A'].map(sku_price_map)
clearance_bundles_df['Price_B'] = clearance_bundles_df['SKU_B'].map(sku_price_map)
clearance_bundles_df['OptimizedBundlePrice'] = clearance_bundles_df['Price_A'] + clearance_bundles_df['Price_B']

sku_to_title = orders_df_copy[['SKU', 'Item title']].drop_duplicates()
title_to_sku = dict(zip(sku_to_title['Item title'], sku_to_title['SKU']))
thematic_bundles_df['SKU_A'] = thematic_bundles_df['Item title A'].map(title_to_sku)
thematic_bundles_df['SKU_B'] = thematic_bundles_df['Item title B'].map(title_to_sku)
thematic_bundles_df['Price_A'] = thematic_bundles_df['SKU_A'].map(sku_price_map)
thematic_bundles_df['Price_B'] = thematic_bundles_df['SKU_B'].map(sku_price_map)
thematic_bundles_df['OptimizedBundlePrice'] = thematic_bundles_df['Price_A'] + thematic_bundles_df['Price_B']





